<h1>1. Regresión Lineal Ordinaria (LSS)</h1>

In [2]:
import numpy as np
import pandas as pd

a) Se elimina id, date y zipcode ya que no aportan informacion relevante en cuanto al precio de las casas

In [3]:
df = pd.read_csv("kc_house_data.csv")
df.drop(['id','date','zipcode',],axis=1,inplace=True)

b) El dataset consta de 18 columnas o atributos, las cuales presentan valores no nulos y enteros o reales, un total de 21.613 instancias de casas.
Todas las casas se encuentran localizadas en King County, esto se refleja en la media de la latitud y longitud (47.560053, -122.213896) y su baja desviasión estándar, la mayoria presenta entre 3 a 4 dormitorios y 1 a 2 baños, los demás datos presentan mayor dispersión por lo que no son muy útiles para generalizar.

In [4]:
df.head()
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 18 columns):
price            21613 non-null float64
bedrooms         21613 non-null int64
bathrooms        21613 non-null float64
sqft_living      21613 non-null int64
sqft_lot         21613 non-null int64
floors           21613 non-null float64
waterfront       21613 non-null int64
view             21613 non-null int64
condition        21613 non-null int64
grade            21613 non-null int64
sqft_above       21613 non-null int64
sqft_basement    21613 non-null int64
yr_built         21613 non-null int64
yr_renovated     21613 non-null int64
lat              21613 non-null float64
long             21613 non-null float64
sqft_living15    21613 non-null int64
sqft_lot15       21613 non-null int64
dtypes: float64(5), int64(13)
memory usage: 3.0 MB


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,47.560053,-122.213896,1986.552492,12768.455652
std,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,0.138564,0.140828,685.391304,27304.179631
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,47.155900,-122.519000,399.000000,651.000000
25%,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,47.678000,-122.125000,2360.000000,10083.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,47.777600,-121.315000,6210.000000,871200.000000


c) Esta parte corresponde al preprocesamiento de los datos, y es utilizado cuando los datos tienen valores muy dispersos (en este caso casi todos exceptuando: 'bedrooms', 'bathrooms', 'lat', 'long'). La motivación detrás de esto radica en que algunos algoritmos, la función objetivo no funciona muy bien sin antes realizar normalización, es decir los datos deben comportarse parecido a una distribución normal estándar.
Luego se aplica función logaritmo al valor del precio ya que lo que nos interesa son los cambios relativos de este valor según los atributos en juego.

In [5]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['price'] = np.log(df['price'])
df_scaled.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
count,21613.000000,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04,2.161300e+04
mean,13.047817,2.367051e-16,-1.788439e-16,2.419652e-16,1.052023e-17,-3.156068e-17,-5.260114e-18,-1.052023e-17,-2.577456e-16,2.524855e-16,1.104624e-16,6.838148e-17,1.914681e-15,1.315028e-18,-3.446262e-14,-3.636843e-14,8.153176e-17,-2.893063e-17
std,0.526685,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00,1.000023e+00
min,11.225243,-3.624404e+00,-2.745920e+00,-1.948891e+00,-3.521759e-01,-9.154270e-01,-8.717263e-02,-3.057595e-01,-3.702668e+00,-5.663344e+00,-1.809494e+00,-6.586810e-01,-2.417383e+00,-2.101284e-01,-2.916795e+00,-2.166543e+00,-2.316325e+00,-4.438052e-01
25%,12.682152,-3.987371e-01,-4.736214e-01,-7.108948e-01,-2.430487e-01,-9.154270e-01,-8.717263e-02,-3.057595e-01,-6.291869e-01,-5.588357e-01,-7.226314e-01,-6.586810e-01,-6.810785e-01,-2.101284e-01,-6.426977e-01,-8.102505e-01,-7.244971e-01,-2.808593e-01
50%,13.017003,-3.987371e-01,1.756067e-01,-1.849914e-01,-1.808075e-01,1.053939e-02,-8.717263e-02,-3.057595e-01,-6.291869e-01,-5.588357e-01,-2.758102e-01,-6.586810e-01,1.360059e-01,-2.101284e-01,8.478232e-02,-1.143518e-01,-2.138280e-01,-1.885636e-01
75%,13.377006,6.764851e-01,5.002207e-01,5.118578e-01,-1.066880e-01,9.365058e-01,-8.717263e-02,-3.057595e-01,9.075535e-01,2.919156e-01,5.091458e-01,6.066704e-01,8.849999e-01,-2.101284e-01,8.512345e-01,6.312541e-01,5.448802e-01,-9.835556e-02
max,15.856731,3.185793e+01,7.641730e+00,1.247807e+01,3.950434e+01,3.714405e+00,1.147149e+01,4.914129e+00,2.444294e+00,4.545672e+00,9.204044e+00,1.023238e+01,1.497813e+00,4.806428e+00,1.570054e+00,6.383070e+00,6.162239e+00,3.144029e+01


In [6]:
print(df.columns)
print(df_scaled['price'][:30])

Index([u'price', u'bedrooms', u'bathrooms', u'sqft_living', u'sqft_lot',
       u'floors', u'waterfront', u'view', u'condition', u'grade',
       u'sqft_above', u'sqft_basement', u'yr_built', u'yr_renovated', u'lat',
       u'long', u'sqft_living15', u'sqft_lot15'],
      dtype='object')
0     12.309982
1     13.195614
2     12.100712
3     13.311329
4     13.142166
5     14.018451
6     12.458775
7     12.583995
8     12.343658
9     12.685408
10    13.403776
11    13.056224
12    12.644328
13    12.899220
14    13.180632
15    13.384728
16    12.886641
17    13.091904
18    12.149502
19    12.345835
20    12.860999
21    14.508658
22    12.560244
23    12.439958
24    12.703813
25    12.358794
26    13.750439
27    13.410545
28    12.989974
29    13.485617
Name: price, dtype: float64


d) Los datos ya se han centrado (paso anterior) por lo que el parametro 'fit_intercept' no se utilizará.
Luego nos interesan todas las columnas exceptuando 'price' ya que solo se quiere incluir los parámetros como los valores de $X$. La linea 4 añade la columna nº18 (luego de eliminar la columna *'price'*) de intercepto, este paso es el mismo realizado en clases en donde el intercepto de la ecuación lineal que representa el comportamiento del precio de acuerdo a los atributos es añadido al dataset o matriz de parámetros $X$ con el fin de encapsular el parámetro desconocido (*$b_0$*) en un solo vector (columna).

Como se mencionó anteriormente, *'fit_intercept'* no se utilizará ya que el intercepto es eliminado de la ecuación lineal e integrado a la matriz de parámetros por lo tanto en los argumentos de la función que implementa la regresión lineal no será necesario. Luego *'normalize'* es ignorado ya que esto se realizo en el paso anterior y *'n_jobs'* es utilizado para dividir el trabajo en multiples CPUs.


In [8]:
import sklearn.linear_model as lm
X = df_scaled.iloc[:,1:] #use .ix instead, in older pandas version //Se consideran todas las filas y columnas menos 'price'
N = X.shape[0] #Cantidad de filas
X.insert(X.shape[1], 'intercept', np.ones(N)) #X.shape[1] = dimensionalidad de datos (18, incluyendo intercepto)
y = df_scaled['price']

#mascara estatica con el 70% de los datos
mascara = np.zeros(len(X))
limit = int(len(X)*0.7)
mascara[:limit] = 1
istrain = mascara == 1
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]

linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)
#predictions_train = linreg.predict(x_train)
#residuals_train = predictions_train - ytrain

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [9]:
linreg.score(Xtest, ytest) #Coeficiente de determinacion R^2

0.76283599664893009

e) Las variables con mayor correlación con la respuesta son *'sqft_above'*, *'sqft_basement'* y *'sqft_living'* ya que presentan el mayor peso absoluto. Luego, un atributo con $z-score$ mayor a 1.960 en valor absoluto es importante a un nivel de significancia del 5%, por lo tanto *'sqft_living'* y *'sqft_above'* serían las variables con mayor influencia, esto es causado ya que son los atributos que mas impacto tienen en el precio final de las casas, al calcular el $z-score$ con 5% de significancia tenemos que con un 95% de confiabilidad los atributos son relevantes, y esto se sostiene hasta con un nivel de significancia del 1% (|z-score| > 2.576).

In [10]:
from scipy.stats import zscore
from scipy.stats import t
df_e = pd.DataFrame(zip(X.columns, linreg.coef_,), columns = ['Atributos', 'Pesos'])
zscore = df_e[df_e.select_dtypes(include=[np.number]).columns].apply(zscore)
df_e.insert(df_e.iloc[:,:].shape[1], "Z-scores", zscore)
df_e

,Atributos,Pesos,Z-scores
0,bedrooms,-1.137666e-02,-0.063335
1,bathrooms,5.664423e-02,-0.063335
2,sqft_living,-1.091202e+12,-3.035989
3,sqft_lot,2.186016e-02,-0.063335
4,floors,3.741344e-02,-0.063335
5,waterfront,3.370060e-02,-0.063335
6,view,4.176399e-02,-0.063335
7,condition,4.571693e-02,-0.063335
8,grade,1.863173e-01,-0.063335
9,sqft_above,9.838574e+11,2.616891


In [ ]:
yhat_test = linreg.predict(Xtest)
mse_test = np.mean(np.power(yhat_test - ytest, 2))
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
mse_cv = 0
for train, val in kf.split(Xm):
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xm[train], ym[train])
yhat_val = linreg.predict(Xm[val])
mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
mse_cv += mse_fold
mse_cv = mse_cv / 10

<h1>2. Selección de Atributos</h1>

In [ ]:
def fss(x, y, names_x, k = 10000):
    p = x.shape[1]-1
    k = min(p, k)
    names_x = np.array(names_x)
    remaining = range(0, p)
    selected = [p]
    current_score = best_new_score = 0.0
    while remaining and len(selected)<=k :
        score_candidates = []
        for candidate in remaining:
            model = lm.LinearRegression(fit_intercept=False)
            indexes = selected + [candidate]
            x_train = x[:,indexes]
            predictions_train = model.fit(x_train, y).predict(x_train)
            residuals_train = predictions_train - y
            mse_candidate = np.mean(np.power(residuals_train, 2))
            score_candidates.append((mse_candidate, candidate))
        score_candidates.sort()
        score_candidates[:] = score_candidates[::-1]
        best_new_score, best_candidate = score_candidates.pop()
        remaining.remove(best_candidate)
        selected.append(best_candidate)
        print "selected = %s ..."%names_x[best_candidate]
        print "totalvars=%d, mse = %f"%(len(indexes),best_new_score)
    return selected

names_regressors = X.columns[:-1] #without intercept
fss(Xm,ym,names_regressors)

<h1>3. Regularización</h1>

<h1>4. Drift</h1>

<h1>5. Detectar enfermedades cardiacas</h1>

In [ ]:
headers = ['age','sex','chest_pain','blood_p','serum','blood_s','electro','max_heart', 'angina','oldpeak','slope',\
           'vessel','thal','normal']
df = pd.read_csv("heart.dat", header=None, names=headers, sep=' ')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

import sklearn.linear_model as lm
X = df_scaled.iloc[:,:] #use .ix instead, in older pandas version //Se consideran todas las filas y columnas menos 'price'
N = X.shape[0] #Cantidad de filas
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['normal']

#mascara estatica con el 70% de los datos
mascara = np.zeros(len(X))
limit = int(len(X)*0.7)
mascara[:limit] = 1
istrain = mascara == 1
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]

linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)

from sklearn.metrics import accuracy_score
print "Score: "%(accuracy_score(y_outlier,y_predict_outlier)